In [3]:
from torch.distributions import constraints
from pyro.distributions.constraints import positive_definite
import numpy as np
import pyro.distributions as dist
import pyro
import scipy as sp
import torch

In [4]:
pyro.enable_validation(True)

In [5]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [6]:
N = 5
D = 3


X = np.random.dirichlet([3, 3, 3], (N, D))
Y = np.argmax(np.sum(X, axis=1), axis=1)

In [10]:
Mu = pyro.param("M_mu", torch.eye(3).double())
Sigma = pyro.param("M_sigma", torch.tensor(np.tile(np.expand_dims(np.eye(3), 0), (3, 1, 1))).double())
M = dist.MultivariateNormal(Mu, Sigma)

NotImplementedError: Cannot transform _PositiveDefinite constraints

In [43]:
M.shape(), M.batch_shape, M.event_shape

(torch.Size([3, 3]), torch.Size([3]), torch.Size([3]))

In [54]:
M_prime = M.sample()
M_prime, np.linalg.eigvals(M_prime), is_pos_def(M_prime)

(tensor([[ 0.2475, -0.7475,  1.0429],
         [-0.6714,  1.4041, -0.2527],
         [ 0.0152, -0.9228,  1.3325]], dtype=torch.float64),
 array([2.2599312 +0.j        , 0.36209341+0.17233274j,
        0.36209341-0.17233274j]),
 True)

## Model

In [31]:
def model(X, Y):
    N, L, Dim = X.shape
    for i in pyro.plate("sequences", N):
        x = X[i]
        y = Y[i]
        M = pyro.sample("M", dist.MultivariateNormal(Mu, Sigma))
        x_i_j = torch.zeros(Dim).double()
        for j in pyro.markov(range(L)):
            print("M", M, M.shape)
            with torch.no_grad():
                print("x_i_j", x_i_j, M @ x_i_j)
                x_i_j = pyro.sample("x_{}_{}".format(i, j), dist.Dirichlet(M @ x_i_j + x[j]))
        y_j = pyro.sample("y_j", dist.Categorical(x_i_j), obs=y)

In [32]:
model(X, Y)

M tensor([[-0.1109,  0.5720, -0.1998],
        [-0.8575,  1.2483, -0.0135],
        [-1.2827, -0.9944,  2.2007]], dtype=torch.float64,
       grad_fn=<AddBackward0>) torch.Size([3, 3])
x_i_j tensor([0., 0., 0.], dtype=torch.float64) tensor([0., 0., 0.], dtype=torch.float64)
M tensor([[-0.1109,  0.5720, -0.1998],
        [-0.8575,  1.2483, -0.0135],
        [-1.2827, -0.9944,  2.2007]], dtype=torch.float64,
       grad_fn=<AddBackward0>) torch.Size([3, 3])
x_i_j tensor([1.8564e-01, 8.1436e-01, 4.1861e-45], dtype=torch.float64) tensor([ 0.4452,  0.8574, -1.0479], dtype=torch.float64)


ValueError: The parameter concentration has invalid values